#### Black and Scholes parameters calibration
The **Implied Volatility** is that value $\\sigma$ that must be inserted into the Black-Scholes (BS) formula in order to retrieve the option price quoted in the market:
$$ BS(S, K, T, r, \sigma) = P  $$
where $S$ is the underlying spot price, $K$ is the strike, $T$ time to maturity, $r$ risk-free interest rate and $P$ the option price quoted in the market. All these quantities are **observable**.,


In [38]:
from functions.BSpricer import BS_pricer

import numpy as np
import pandas as pd
import scipy as scp
import scipy.stats as ss
import scipy.optimize as scpo


Let's retrieve the historic prices for european call and put options starting from 2016-01-20 and expiring 1 year later.

In [39]:
df_call = pd.read_csv('data/options_spx_call_2016.csv')
df_put = pd.read_csv('data/options_spx_put_2016.csv')

df_call['C_Midpoint'] = abs(df_call['C_BID'] + df_call['C_ASK'] )/ 2
df_put['P_Midpoint'] = abs(df_put['P_BID'] + df_put['P_ASK'] )/ 2

r = 0.1
sigma = 0.5
S0 = df_call.iloc[0]['UNDERLYING_LAST']
T = 1
call_strikes = df_call['STRIKE']    # array of K
put_strikes = df_put['STRIKE']
exercise = 'european'

call_prices = df_call['C_Midpoint']
put_prices = df_put['P_Midpoint']

#print(df_call)

Now let's initialize an object of class BS_pricer which is able to find the theoretical price of the options, given the parameters.

In [40]:
BS = BS_pricer(S0=S0, r=r, sigma=sigma, ttm=T, exercise=exercise, K=None, type_o='call')

BS_call_prices = np.zeros_like(call_strikes, dtype=float)
for i, K in enumerate(call_strikes):
    BS_call_prices[i] = BS.closed_formula(K)

print(BS_call_prices)


[519.26777644 506.92447997 494.84144177 483.01557869 471.44368111
 460.12242595 449.04838896 438.21805656 427.62783697 417.27407075
 407.15304077 397.26098163 387.5940885  378.14852541]


PUT PRICES:

In [41]:
print(df_put)

BS = BS_pricer(S0=S0, r=r, sigma=sigma, ttm=T, exercise=exercise, K=None, type_o='put')

BS_put_prices = np.zeros_like(put_strikes, dtype=float)
for i, K in enumerate(put_strikes):
    BS_put_prices[i] = BS.closed_formula(K)

print(BS_put_prices)

    QUOTE_DATE  UNDERLYING_LAST EXPIRE_DATE   P_BID   P_ASK  STRIKE   
0   2016-01-20          1859.48  2017-01-20  108.51  111.60  1700.0  \
1   2016-01-20          1859.48  2017-01-20  116.50  119.70  1725.0   
2   2016-01-20          1859.48  2017-01-20  124.40  127.61  1750.0   
3   2016-01-20          1859.48  2017-01-20  133.01  136.30  1775.0   
4   2016-01-20          1859.48  2017-01-20  142.29  145.61  1800.0   
5   2016-01-20          1859.48  2017-01-20  152.01  155.41  1825.0   
6   2016-01-20          1859.48  2017-01-20  162.39  165.89  1850.0   
7   2016-01-20          1859.48  2017-01-20  173.29  176.81  1875.0   
8   2016-01-20          1859.48  2017-01-20  184.89  188.51  1900.0   
9   2016-01-20          1859.48  2017-01-20  197.10  200.70  1925.0   
10  2016-01-20          1859.48  2017-01-20  210.00  213.70  1950.0   
11  2016-01-20          1859.48  2017-01-20  223.59  227.39  1975.0   
12  2016-01-20          1859.48  2017-01-20  237.90  241.80  2000.0   
13  20

In [42]:
def implied_volatility(price, S0, K, T, r, type_o, method, disp=True ):
    """ Returns Implied volatility
        methods:  fsolve (default) or brent
    """

    def obj_fun(vol):
        return price - BS.BlackScholes(type_o=type_o, S0=S0, K=K, ttm=T, r=r, sigma=vol)

    if method == 'brent':
        x, r = scpo.brentq( obj_fun, a=1e-15, b=500, full_output=True)
        if r.converged:
            return x

    if method =='fsolve':
        X0 = [0.03, 0.08, 0.1, 0.15, 0.2, 0.3, 0.5, 1, 2]   # set of initial guess points for imp.vol.
        for x0 in X0:
            x, _, solved, _ = scpo.fsolve(obj_fun, x0, full_output=True, xtol=1e-8)
            if solved == 1:
                return x[0]

    if method == 'newton':
        x0 = 0.3

        def obj_fun_squared(vol):
            return obj_fun(vol)**2

        # print('[σ] = ', x0, "Objective Function value: ", obj_fun_squared(x0))
        result = scpo.fmin(obj_fun_squared, x0, ftol=1e-10, full_output=0, disp=0)
        return result[0]


    if disp:
        return -1

Let's now compute the implied volatilities from the true market prices *call_prices*.

In [43]:
IVs = []
for i in range(len(call_prices)):
    IVs.append(implied_volatility(call_prices[i], S0=S0, K=call_strikes[i], T=T, r=r, type_o='call', method='newton') )

print(IVs)


[0.014999999999999208, 0.014999999999999208, 0.014999999999999208, 0.014999999999999208, 0.014999999999999208, 0.014765624999999208, 0.007499999999999188, 0.007499999999999188, 0.007499999999999188, 0.007499999999999188, 0.0037499999999991776, 0.04645385742187429, 0.057302570343016904, 0.0637830448150628]


In [44]:
IVs = []
for i in range(len(put_prices)):
    IVs.append(implied_volatility(put_prices[i], S0=S0, K=put_strikes[i], T=T, r=r, type_o='put', method='fsolve') )

print(IVs)


[0.35229363616294335, 0.35188275048143236, 0.3506820060204052, 0.35017099337544816, 0.35014355407871467, 0.3502933001043605, 0.35092279692388245, 0.3517300535108708, 0.3530935523924029, 0.3547251036727889, 0.35691040121733497, 0.3595598971892729, 0.3627175829845052, 0.36647819300704754]
